In [1]:
import numpy as np
import gc
import energyflow as ef
import pandas as pd
from energyflow.archs import EFN
import matplotlib.pyplot as plt
import awkward as ak
from energyflow.utils import to_categorical


In [2]:
from energyflow.archs import PFN

In [3]:
qq_test = ak.from_parquet("my_qqcand_test.parquet")

In [4]:
qq_test

<Array [{reco_cand_p4s: [{...}], ...}, ...] type='6366715 * {reco_cand_p4s:...'>

reading the angular 4 momentums created in data_intro

In [5]:
#reading data
zh = ak.from_parquet(r"my_zhcand.parquet")
z = ak.from_parquet(r"C:\Users\yusuf\OneDrive\Masaüstü\mlforphyscis\my_zcand.parquet")
qq = ak.from_parquet(r"C:\Users\yusuf\OneDrive\Masaüstü\mlforphyscis\my_qqcand.parquet")

In [6]:
zh_pid = ak.from_parquet(r"C:\Users\yusuf\OneDrive\Masaüstü\mlforphyscis\my_pid_zh.parquet")
z_pid = ak.from_parquet("my_pid_z.parquet")
qq_pid = ak.from_parquet("my_pid_qq.parquet")

In [7]:
zh

<Array [[{rho: 21.6, phi: ..., ...}, ...], ...] type='2087907 * var * Momen...'>

padding and filling the data to even jet sizes with 0 particles
also some preprocessing

In [8]:
import itertools as IT
#function that separates data and formats it for EFN returns the pt and phat values
def to_efn(data):
    data_phat = data[:][["eta","phi"]]
    data_rho = data[:]["rho"]
    data_rho_padded = ak.pad_none(data_rho, target=80, clip = True)
    data_rho_padded_filled = ak.fill_none(data_rho_padded, 0)
    data_phat_padded = ak.pad_none(data_phat,target=80, clip =True)
    data_phat_padded_filled = ak.fill_none(data_phat_padded, value = {"eta": 0, "phi": 0})

    test = ak.to_numpy(data_phat_padded_filled)
    data_phat = test.view(((float, len(test.dtype.names))))

    data_rho_padded_filled = np.asarray(data_rho_padded_filled).astype('float32')
    x = 0
    for jet_rho, jet_phat in zip(data_rho_padded_filled, data_phat):
        jet_rho /= jet_rho.sum()
        phi_average = np.average(jet_phat,axis=0, weights=jet_rho)
        jet_phat -=phi_average

    return  data_phat, data_rho_padded_filled

In [9]:
qq_pid = ak.fill_none(ak.pad_none(qq_pid, target=80, clip=True),0)
zh_pid = ak.fill_none(ak.pad_none(zh_pid, target=80, clip=True),0)
z_pid = ak.fill_none(ak.pad_none(z_pid, target=80, clip=True),0)

In [10]:
qq_phat, qq_rho = to_efn(qq)
zh_phat , zh_rho =to_efn(zh)
z_phat , z_rho =to_efn(z)

In [11]:
del qq, zh, z
gc.collect()

129

In [12]:
rhos = np.vstack((qq_rho, zh_rho, z_rho))#combining the 3 data sources

In [13]:
phats = np.vstack((qq_phat, zh_phat, z_phat))

target values are based on data sources so I can just open some arrays of same size to use as label

In [14]:
zh_target = np.ones(len(zh_rho)) #our normal target data is tau decay mode yet this is needlesly complicated
z_target = np.ones(len(z_rho)) #it has 17 differenet values based on the decaymode which is unlike the 1 and 0 we want
qq_target = np.zeros(len(qq_rho))#we know our resources corrospond to the targey values so i will just use this

In [15]:
del qq_phat, qq_rho, zh_phat, zh_rho, z_phat, z_rho
gc.collect()

0

In [16]:
Yzh = to_categorical(zh_target, num_classes=2)#number of classes represent background or signal both having 0 and 1
Yz = to_categorical(z_target, num_classes=2)
Yqq = to_categorical(qq_target, num_classes=2)

In [17]:
Y = np.vstack((Yqq, Yzh, Yz))

In [18]:
del Yzh,Yz,Yqq, qq_target, zh_target, z_target
gc.collect()

0

In [19]:
phats

array([[[-5.28339658e-03, -1.86498190e-01],
        [-2.71266722e-02, -2.37941057e-01],
        [-1.19913056e-02, -1.85120306e-01],
        ...,
        [-1.21919647e+00,  2.83643672e+00],
        [-1.21919647e+00,  2.83643672e+00],
        [-1.21919647e+00,  2.83643672e+00]],

       [[ 7.13921796e-02, -1.26907055e-02],
        [-4.96455707e-02,  9.29890300e-02],
        [-2.78922896e-02, -1.41268417e-01],
        ...,
        [ 1.08857455e+00,  2.67853409e+00],
        [ 1.08857455e+00,  2.67853409e+00],
        [ 1.08857455e+00,  2.67853409e+00]],

       [[ 2.07151697e-02,  2.57667783e-02],
        [-1.26292540e-01,  2.32466295e-02],
        [ 3.12081838e-01, -1.40202175e-01],
        ...,
        [-6.63359465e-03,  9.87806753e-02],
        [-6.63359465e-03,  9.87806753e-02],
        [-6.63359465e-03,  9.87806753e-02]],

       ...,

       [[ 0.00000000e+00,  0.00000000e+00],
        [-6.28031118e-03,  2.51240529e-02],
        [-6.28031118e-03,  2.51240529e-02],
        ...,
     

In [20]:
from energyflow.utils import data_split
(z_train, z_val, z_test,
 p_train, p_val, p_test,
 Y_train, Y_val, Y_test) = data_split(rhos, phats, Y, val=0.4, test=0.1, shuffle = True)


In [21]:
np.save("z_train.npy", z_train)
np.save("z_val.npy", z_val)
np.save("p_train.npy", p_train)
np.save("p_val.npy", p_val)
np.save("Y_train.npy", Y_train)
np.save("Y_val.npy", Y_val)


model

In [ ]:
from energyflow.archs import PFN
Phi_sizes, F_sizes = [100, 100, 128], [100, 100, 100]
pfn = PFN(input_dim=2, Phi_sizes=Phi_sizes, F_sizes=F_sizes)
# train model
pfn.fit(p_train, Y_train,
        epochs=5,
        batch_size=500, validation_data=(p_val, Y_val),
        verbose=1)
#fit



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, None, 2)]            0         []                            
                                                                                                  
 tdist_0 (TimeDistributed)   (None, None, 100)            300       ['input[0][0]']               
                                                                                                  
 activation (Activation)     (None, None, 100)            0         ['tdist_0[0][0]']             
                                                                                                  
 tdist_1 (TimeDistributed)   (None, None, 100)            10100     ['activation[0][0]']          
                                                                                            

MemoryError: Unable to allocate 4.61 GiB for an array with shape (3864632, 80, 2) and data type float64

: 

test function with prongs

In [12]:
#these are functions for testing, they take the test data format it with to_efn do the prediction, then filter the predictions according to prongs and give out a dictionary
from sklearn.metrics import accuracy_score
#prong is a measure for the number of hadrons in the decay
prong_1 = [0,1,2,3,4]
prong_2 = [5,6,7,8,9]
prong_3 = [10,11,12,13,14]
prong_other = [15]
def from_categorical(Y):
    a = []
    for i in range(len(Y)):
        if Y[i][0]>Y[i][1]:
            a.append(0)
        else:
            a.append(1)
    return a
def test_result(my_cand_test, a):
    _test_phi, _test_rho = to_efn(my_cand_test["4momentum"])
    _pred = pfn.predict([_test_rho, _test_phi])
    if a == 0:
        _true = np.zeros(len(_test_rho))
    else:
        _true = np.ones(len(_test_rho))
    _pred2 = from_categorical(_pred)
    df = pd.DataFrame()
    df["z_true"] = _true
    df["z_pred"] = _pred2
    df["decay_mode"] = ak.to_numpy(my_cand_test["gen_jet_tau_decaymode"])
    df_prong_1 = df[df["decay_mode"].isin(prong_1)]
    df_prong_2 = df[df["decay_mode"].isin(prong_2)]
    df_prong_3 = df[df["decay_mode"].isin(prong_3)]
    df_prong_other = df[df["decay_mode"].isin(prong_other)]
    b = [df_prong_1, df_prong_2, df_prong_3, df_prong_other]
    c = {}
    for i,x in enumerate(b):
        if len(x) != 0:
            acc = accuracy_score(y_true=x["z_true"], y_pred=x["z_pred"])
            c[f"prong{i+1}"] = acc
    return df, c

In [13]:
qq_test = ak.from_parquet(r"C:\Users\yusuf\OneDrive\Masaüstü\ml for physcis\my_qqcand_test.parquet")



In [14]:
qq_test = qq_test[["gen_jet_tau_decaymode", "4momentum"]]
zh_test = ak.from_parquet("zh_test.parquet")


In [15]:
(qq_train, qq_val, qq_test,) = data_split(qq_test, val=0.1, test=0.4, shuffle = True)#have to do a data split for qq because of memory issues

In [16]:
df, acc = test_result(qq_test, 0)
#acc_dict_z = test_result(z_test, 1)
#acc_dict_z = test_result(z_test, 1)

79584/79584 [==============================] - 302s 4ms/step


In [17]:
accuracy_score(y_pred=df["z_pred"], y_true=df["z_true"])

0.689728533474484

In [18]:
acc_dict_zh

NameError: name 'acc_dict_zh' is not defined

In [37]:
acc_dict_z

{'prong0': 0.8571370696083919,
 'prong2': 0.8086259944836651,
 'prong3': 0.80640589569161}